## Set Global Seed 

In [1]:
import os
import random
import numpy as np
import torch
import transformers

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 260615
set_all_seeds(seed)

print("The global seed " + str(seed))

The global seed 260615


## Hyperparameters

In [2]:
# LANGUAGE

_LANGUAGE_         = 'es'

In [3]:
# MODEL CLASSIFICATION

_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-uncased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig(reduction_factor = 256)
_MAX_SEQ_LEN_      = 128

In [4]:
# TRAIN

_OUTPUT_DIR_       = 'checkPointsFE'
_LOGGING_STEPS_    = 50
_NUM_AUTHORS_      = 256
_K_FOLD_CV_        = 5
_NO_GPUS_          = 1
_BATCH_SIZE_       = int(32 / _NO_GPUS_)
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-4

# PREDICTIONS

_DATASET_          = 'PAN17'
_PRED_DIR_         = 'FE_5tweet'

## Other parameters

In [5]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 'male':   1}
varietyEN_dict = {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}
varietyES_dict = {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}  

genderEN_hip  = {0: 'I’m a female', 1: 'I’m a male'}
genderES_hip  = {0: 'Mi nombre es María', 1: 'Mi nombre es José'}

In [6]:
# SET LANGUAGE DICTIONARIES

if _LANGUAGE_ == 'en':
    gender_hip   = genderEN_hip
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    gender_hip   = genderES_hip
    variety_dict = varietyES_dict

In [7]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer, PretrainedConfig

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

## Datasets

In [8]:
from tools.DataLoaders import BasePAN17

baseTrain  = BasePAN17(Dir             = 'data/2017',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = 1,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data/2017',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = 1,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 419998


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 280000



In [9]:
crossVal_splits = []

for val_idx in range(_K_FOLD_CV_):
    
    authors_train, authors_val = baseTrain.cross_val(_K_FOLD_CV_, val_idx, _NUM_AUTHORS_)
    
    crossVal_splits.append( (authors_train, authors_val) )

In [10]:
from tools.DataLoaders import BasePAN17

baseTrain  = BasePAN17(Dir             = 'data/2017',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir              = 'data/2017',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 56000



In [11]:
from tools.DataLoaders import DatasetPAN17

Test = DatasetPAN17(baseTest, 'gender')

In [12]:
crossVal_splits[0][0]

['bafc7670beae172539e66c48858d8a',
 '70250236e5aae9df16d3d081b24f8c52',
 '4230dfe217cc7d667e6a21bff301263d',
 'b1be8bd39d7a3be53d2e37a654db05d8',
 '8690c40c4fea5fdb6ad1c030cafca7f',
 'eb3d30f6a613d63c495faf276ce8af20',
 'aeef24da173bad8c8589593068d12e2b',
 '3fccf54ed8ff8ebf3e09b27178c3eda6',
 '5341c6e38c6ff6d6a3dc11eac05a08de',
 'f8707cb24018d27812ad57586e45bf4e',
 '351e949f234bd6223b9921b3522a90ce',
 'bd2c8464416d5a45cffc1a6ae7db12de',
 '2e5946874ebd8880ac4356c5491d5f85',
 '78c318d74686fee6dea0f10d0fd805f2',
 'cd30f8d42568f41737f63b7fe3d74206',
 '1fb9ec36c0d6afecb0bcdd74dc1140c8',
 '10880d19d7346373a42b18e505bbc4d5',
 '534bfd160e0b7a8659e5854dc8deb795',
 '32d2ef7c31d22593e24a9cc59c00698c',
 '35aa53616736e99b24d0190b114873ab',
 '6c085614b68ce2c31216b4ca45b50131',
 '7c830bd1093b26c192cb0b307303803a',
 '4f3d4bd16d919fdab790f073bc71fefc',
 'de9f0a8cf6ec53f7d0e907b19d92fcab',
 '4ffefe7a85e2acecfe321208aab3c7ca',
 'b90c6c58d2dd8602765bad6df8c4a732',
 '5e7290bad2b0f904247e9e7583010a7d',
 'a2

## Training

In [13]:
from transformers import TrainingArguments

samples = 2 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 10,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [14]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = 'gender'

f1s_soft = []
f1s_hard = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossVal(baseTrain, authors_train, task)
    Val   = DatasetCrossVal(baseTrain, authors_val  , task)
    
    
    # initialize model ---------------------------------------------------
    
    model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = 2)
    
    for name, param in model.named_parameters():
        if 'classifier' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
    
    # create trainer and train -------------------------------------------
        
    trainer = Trainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    results            = trainer.predict(Val)
    author_predictions = compute_author_predictions(Val, results.predictions, 'gender', 2)
    
    
    # report metrics -----------------------------------------------------
    
    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )

    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])
     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
    


Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.we

Step,Training Loss
320,0.560600
640,0.535200
960,0.526700
1280,0.517200
1600,0.512800
1920,0.513400
2240,0.506300
2560,0.497600
2880,0.492800
3200,0.490600


Saving model checkpoint to checkPointsFE/checkpoint-500
Configuration saved in checkPointsFE/checkpoint-500/config.json
Model weights saved in checkPointsFE/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1000
Configuration saved in checkPointsFE/checkpoint-1000/config.json
Model weights saved in checkPointsFE/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1500
Configuration saved in checkPointsFE/checkpoint-1500/config.json
Model weights saved in checkPointsFE/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2000
Configuration saved in checkPointsFE/checkpoint-2000/config.json
Model weights saved in checkPointsFE/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2500
Configuration saved in checkPointsFE/checkpoint-2500/config.json
Model weights saved in checkPointsFE/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to checkPoints

100%|█████████████████████████████████████████████████████████████████████████████████| 840/840 [00:38<00:00, 21.56it/s]


Results with split 1:

soft voting:
               precision    recall  f1-score   support

           0     0.7264    0.7524    0.7392       420
           1     0.7432    0.7167    0.7297       420

    accuracy                         0.7345       840
   macro avg     0.7348    0.7345    0.7344       840
weighted avg     0.7348    0.7345    0.7344       840
 

hard voting:
               precision    recall  f1-score   support

           0     0.7105    0.7714    0.7397       420
           1     0.7500    0.6857    0.7164       420

    accuracy                         0.7286       840
   macro avg     0.7303    0.7286    0.7281       840
weighted avg     0.7303    0.7286    0.7281       840



...

loading configuration file https://huggingface.co/pysentimiento/robertuito-base-uncased/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/5212cb9b5b32726fce956daa9a21ee0a0c2b6e54c54d1af58c678217d85f8143.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "

Step,Training Loss
320,0.581300
640,0.558000
960,0.540500
1280,0.541400
1600,0.532700
1920,0.524700
2240,0.525200
2560,0.517400
2880,0.517500
3200,0.510600


Saving model checkpoint to checkPointsFE/checkpoint-500
Configuration saved in checkPointsFE/checkpoint-500/config.json
Model weights saved in checkPointsFE/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1000
Configuration saved in checkPointsFE/checkpoint-1000/config.json
Model weights saved in checkPointsFE/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1500
Configuration saved in checkPointsFE/checkpoint-1500/config.json
Model weights saved in checkPointsFE/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2000
Configuration saved in checkPointsFE/checkpoint-2000/config.json
Model weights saved in checkPointsFE/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2500
Configuration saved in checkPointsFE/checkpoint-2500/config.json
Model weights saved in checkPointsFE/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to checkPoints

100%|█████████████████████████████████████████████████████████████████████████████████| 840/840 [00:29<00:00, 28.25it/s]

Results with split 5:

soft voting:
               precision    recall  f1-score   support

           0     0.7190    0.7857    0.7509       420
           1     0.7638    0.6929    0.7266       420

    accuracy                         0.7393       840
   macro avg     0.7414    0.7393    0.7387       840
weighted avg     0.7414    0.7393    0.7387       840
 

hard voting:
               precision    recall  f1-score   support

           0     0.7119    0.8119    0.7586       420
           1     0.7812    0.6714    0.7222       420

    accuracy                         0.7417       840
   macro avg     0.7465    0.7417    0.7404       840
weighted avg     0.7465    0.7417    0.7404       840



In [15]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])

Soft results:  [0.7344391281233387, 0.7452149854862119, 0.7101519245815648, 0.7575743789727192, 0.7387225013102223]

Hard results:  [0.7280719689225106, 0.746298772472878, 0.7106220095693779, 0.7511111111111111, 0.7403858890017219]


Soft statistics: 
	[avg, std]: [0.7372205836948115, 0.015626208753411898]

Hard statistics: 
	[avg, std]: [0.7352979502155199, 0.014547014273443326]


## Test the best split

In [16]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal, DatasetPAN17
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = 'gender'

split = 3

# loaders for current split ------------------------------------------
    
authors_train, authors_val = crossVal_splits[split]

Train = DatasetCrossVal(baseTrain, authors_train, task)
Val   = DatasetCrossVal(baseTrain, authors_val  , task)


# initialize model ---------------------------------------------------
    
model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = 2)

for name, param in model.named_parameters():
    if 'classifier' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# create trainer and train -------------------------------------------

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = Train,
)
trainer.args._n_gpu = _NO_GPUS_

trainer.train()


# get predictions ----------------------------------------------------

results            = trainer.predict(Test)
author_predictions = compute_author_predictions(baseTest, results.predictions, 'gender', 2)


# report metrics -----------------------------------------------------

report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

print("Results with split " + str(split + 1) + ":\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])


loading configuration file https://huggingface.co/pysentimiento/robertuito-base-uncased/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/5212cb9b5b32726fce956daa9a21ee0a0c2b6e54c54d1af58c678217d85f8143.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "

Step,Training Loss
320,0.578400
640,0.548600
960,0.542600
1280,0.534200
1600,0.525000
1920,0.525200
2240,0.522600
2560,0.515800
2880,0.508100
3200,0.512100


Saving model checkpoint to checkPointsFE/checkpoint-500
Configuration saved in checkPointsFE/checkpoint-500/config.json
Model weights saved in checkPointsFE/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1000
Configuration saved in checkPointsFE/checkpoint-1000/config.json
Model weights saved in checkPointsFE/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-1500
Configuration saved in checkPointsFE/checkpoint-1500/config.json
Model weights saved in checkPointsFE/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2000
Configuration saved in checkPointsFE/checkpoint-2000/config.json
Model weights saved in checkPointsFE/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to checkPointsFE/checkpoint-2500
Configuration saved in checkPointsFE/checkpoint-2500/config.json
Model weights saved in checkPointsFE/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to checkPoints

100%|███████████████████████████████████████████████████████████████████████████████| 2800/2800 [05:49<00:00,  8.00it/s]

Results with split 4:

soft voting:
               precision    recall  f1-score   support

           0     0.7435    0.8157    0.7779      1400
           1     0.7959    0.7186    0.7553      1400

    accuracy                         0.7671      2800
   macro avg     0.7697    0.7671    0.7666      2800
weighted avg     0.7697    0.7671    0.7666      2800
 

hard voting:
               precision    recall  f1-score   support

           0     0.7225    0.8293    0.7722      1400
           1     0.7997    0.6814    0.7358      1400

    accuracy                         0.7554      2800
   macro avg     0.7611    0.7554    0.7540      2800
weighted avg     0.7611    0.7554    0.7540      2800

